In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [15]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://thewaltdisneycompany.com/investor-relations/"

response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.text, 'html.parser')

pdf_links = soup.find_all('a', class_='pdf-file')

save_dir = "disney_reports"
os.makedirs(save_dir, exist_ok=True)

# Choose what to download: 'annual' or 'quarterly'
report_type = 'quarterly' 

for link in pdf_links:
    pdf_url = link.get('href')
    if pdf_url.endswith('.pdf'):
        link_text = link.get_text(strip=True).lower() 
        
        if (report_type == 'annual' and 'annual report' in link_text) or \
           (report_type == 'quarterly' and ('q1' in link_text or 'q2' in link_text or 'q3' in link_text or 'q4' in link_text or 'earnings report' in link_text)):
            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()
            
            file_name = pdf_url.split('/')[-1]
            save_path = os.path.join(save_dir, file_name)
            
            with open(save_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {file_name}")

print(f"All {report_type} have been downloaded.")


Downloaded q1-fy24-earnings.pdf
Downloaded q4-fy23-earnings.pdf
Downloaded q3-fy23-earnings.pdf
Downloaded q2-fy23-earnings.pdf


KeyboardInterrupt: 

In [11]:
pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/eb/be/cce6fc626c898458b304dc19864f77ae635b9f5c09a41e8e9d1363486d1a/PyMuPDF-1.23.25-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.22 from https://files.pythonhosted.org/packages/7f/40/f02986ac1c9f580bdf90e405ae29b216fd6b481be04ec7b9a70b6b502cf1/PyMuPDFb-1.23.22-py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 8.6 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import fitz
import os

def extract_section(pdf_path, start_keywords, end_keywords):
    """
    Extracts sections from a PDF based on start and end keywords.

    :param pdf_path: Path to the PDF file.
    :param start_keywords: List of keywords indicating the start of the section.
    :param end_keywords: List of keywords indicating the end of the section.
    :return: Text of the extracted section.
    """
    doc = fitz.open(pdf_path)
    text = ""
    capture = False
    
    for page in doc:
        page_text = page.get_text()
        if any(keyword.lower() in page_text.lower() for keyword in start_keywords) and not capture:
            capture = True
        if any(keyword.lower() in page_text.lower() for keyword in end_keywords) and capture:
            text += page_text
            break
        if capture:
            text += page_text
    
    doc.close()
    return text

In [16]:
pdf_dir = "disney_reports"
balance_sheet_keywords = ["balance sheet"]
cash_flows_keywords = ["cash flows", "statement of cash flows"]
income_statement_keywords = ["income statement", "statement of operations"]

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        print(f"Extracting from {pdf_file}...")
        
        balance_sheet_section = extract_section(pdf_path, balance_sheet_keywords, cash_flows_keywords)
        print("Balance Sheet Section:")
        print(balance_sheet_section)

Extracting from q1-fy24-earnings.pdf...
Balance Sheet Section:
THE WALT DISNEY COMPANY
CONDENSED CONSOLIDATED BALANCE SHEETS
(unaudited; $ in millions, except per share data)
December 30,
2023
September 30,
2023
ASSETS
Current assets
Cash and cash equivalents
$ 
7,192   
$ 
14,182   
Receivables, net
 
14,115 
 
12,330 
Inventories
 
1,954 
 
1,963 
Content advances
 
1,409 
 
3,002 
Other current assets
 
1,301 
 
1,286 
Total current assets
 
25,971 
 
32,763 
Produced and licensed content costs
 
32,725 
 
33,591 
Investments
 
3,084 
 
3,080 
Parks, resorts and other property
Attractions, buildings and equipment
 
72,096 
 
70,090 
Accumulated depreciation
 
(43,575) 
 
(42,610) 
 
28,521 
 
27,480 
Projects in progress
 
5,618 
 
6,285 
Land
 
1,182 
 
1,176 
 
35,321 
 
34,941 
Intangible assets, net
 
12,639 
 
13,061 
Goodwill
 
77,066 
 
77,067 
Other assets
 
10,968 
 
11,076 
Total assets
$ 
197,774 
$ 
205,579 
LIABILITIES AND EQUITY
Current liabilities
Accounts payable and